In [2]:
import pandas as pd
import numpy as np
import json

filename = 'skill_builder_data_corrected.csv'
df = pd.read_csv(filename, encoding='ISO-8859-1', low_memory=False)
df = df[(df['original'] == 1) & (df['attempt_count'] == 1) & ~(df['skill_name'].isnull())]

In [7]:
def generate_datasets():
    users_list = df['user_id'].unique()
    skill_list = df['skill_name'].unique()
    
    skill_dict = dict(zip(skill_list, np.arange(len(skill_list), dtype='int32') + 1))
    response_list = []
    skill_list = []
    assistment_list = []
    
    counter = 0
    for user in users_list:
        sub_df = df[df['user_id'] == user]
        if len(sub_df) > 100:
            first_hundred = sub_df.iloc[0:100]
            response_df = pd.DataFrame(index=[counter], columns=['student_id']+['r'+str(i) for i in range(100)])
            skill_df = pd.DataFrame(index=[counter], columns=['student_id']+['s'+str(i) for i in range(100)])
            assistment_df = pd.DataFrame(index=[counter], columns=['student_id']+['a'+str(i) for i in range(100)])
            
            response_df.iloc[0, 0] = first_hundred.iloc[0]['user_id']
            skill_df.iloc[0, 0] = first_hundred.iloc[0]['user_id']
            assistment_df.iloc[0, 0] = first_hundred.iloc[0]['user_id']
            for i in range(100):
                response_df.iloc[0, i+1] = first_hundred.iloc[i]['correct']
                skill_df.iloc[0, i+1] = skill_dict[first_hundred.iloc[i]['skill_name']]
                assistment_df.iloc[0, i+1] = first_hundred.iloc[i]['assistment_id']
            counter += 1
            response_list.append(response_df)
            skill_list.append(skill_df)
            assistment_list.append(assistment_df)
    
    response_df = pd.concat(response_list)
    skill_df = pd.concat(skill_list)
    assistment_df = pd.concat(assistment_list)
    
    return skill_dict, response_df, skill_df, assistment_df
    
skill_dict, response_df, skill_df, assistment_df = generate_datasets()

with open('skill_dict.json', 'w', encoding='utf-8') as f:
    to_dump_dict = {}
    for key, value in skill_dict.items():
        to_dump_dict[key] = str(value)
    json.dump(to_dump_dict, f)
response_df.to_csv('correct.tsv', sep='\t')
skill_df.to_csv('skill.tsv', sep='\t')
assistment_df.to_csv('assistment_id.tsv', sep='\t')
print('Done')

Done


In [3]:
response_df = pd.read_csv('correct.tsv', sep='\t').drop('Unnamed: 0', axis=1)
skill_df = pd.read_csv('skill.tsv', sep='\t').drop('Unnamed: 0', axis=1)
assistment_df = pd.read_csv('assistment_id.tsv', sep='\t').drop('Unnamed: 0', axis=1)
skill_dict = {}
with open('skill_dict.json', 'r', encoding='utf-8') as f:
    loaded = json.load(f)
    for k, v in loaded.items():
        skill_dict[k] = int(v)

skill_num = len(skill_dict) + 1 # including 0

def one_hot(skill_matrix, vocab_size):
    '''
    params:
        skill_matrix: 2-D matrix (student, skills)
        vocal_size: size of the vocabulary
    returns:
        a ndarray with a shape like (student, sequence_len, vocab_size)
    '''
    seq_len = skill_matrix.shape[1]
    result = np.zeros((skill_matrix.shape[0], seq_len, vocab_size))
    for i in range(skill_matrix.shape[0]):
        result[i, np.arange(seq_len), skill_matrix[i]] = 1.
    return result

def dkt_one_hot(skill_matrix, response_matrix, vocab_size):
    seq_len = skill_matrix.shape[1]
    skill_response_array = np.zeros((skill_matrix.shape[0], seq_len, 2 * vocab_size))
    masking_array = np.zeros((skill_matrix.shape[0], seq_len, 2 * vocab_size))
    for i in range(skill_matrix.shape[0]):
        skill_response_array[i, np.arange(seq_len), 2 * skill_matrix[i] + response_matrix[i]] = 1.
        masking_array[i, np.arange(seq_len), 2 * skill_matrix[i]] = 1.
        masking_array[i, np.arange(seq_len), 2 * skill_matrix[i] + 1] = 1.
    return skill_response_array, masking_array

def preprocess(skill_df, response_df, skill_num):
    skill_matrix = skill_df.iloc[:, 1:].values
    response_array = response_df.iloc[:, 1:].values
    skill_array = one_hot(skill_matrix, skill_num)
    skill_response_array, masking_array = dkt_one_hot(skill_matrix, response_array, skill_num)
    return skill_array, response_array, skill_response_array, masking_array
    

skill_array, response_array, skill_response_array, masking_array = preprocess(skill_df, response_df, skill_num)

In [19]:
import keras
from keras.layers import Input, Dense, LSTM, TimeDistributed, Lambda, multiply
from keras.models import Model
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K

def build_skill2skill_model(input_shape, lstm_dim=32, dropout=0.0):
    input = Input(shape=input_shape)
    lstm = LSTM(lstm_dim, 
                return_sequences=True, 
                dropout=dropout)(input)
    output = TimeDistributed(Dense(input_shape[-1], activation='softmax'), name='probability')(lstm)
    model = Model(inputs=[input], outputs=[output])
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.0)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def reduce_dim(x):
    x = K.max(x, axis=-1, keepdims=True)
    return x

def build_dkt_model(input_shape, lstm_dim=32, dropout=0.0):
    input_skills = Input(shape=input_shape)
    lstm = LSTM(lstm_dim, 
                return_sequences=True, 
                dropout=dropout)(input_skills)
    dense = TimeDistributed(Dense(input_shape[-1], activation='sigmoid'))(lstm)
    
    skill_next = Input(shape=input_shape)
    merged = multiply([dense, skill_next])
    reduced = Lambda(reduce_dim, output_shape=(input_shape[0], 1))(merged)
    
    model = Model(inputs=[input_skills, skill_next], outputs=[reduced])
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.0)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

print('skill2skill')
skill2skill_model = build_skill2skill_model((99, skill_num), lstm_dim=64)

print('dkt')
dkt_model = build_dkt_model((99, 2 * skill_num), lstm_dim=64)
    

skill2skill
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 99, 111)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 99, 64)            45056     
_________________________________________________________________
probability (TimeDistributed (None, 99, 111)           7215      
Total params: 52,271
Trainable params: 52,271
Non-trainable params: 0
_________________________________________________________________
dkt
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_16 (InputLayer)            (None, 99, 222)       0                                            
___________________________________________________________________________________

In [20]:
# train skill2skill
skill2skill_model.fit(skill_array[:, 0:-1], 
                      skill_array[:, 1:],
                      epochs=20, 
                      batch_size=32, 
                      shuffle=True,
                      validation_split=0.2)

Train on 467 samples, validate on 117 samples
Epoch 1/20
467/467 [==============================] - 3s - loss: 4.6455 - acc: 0.1293 - val_loss: 4.5946 - val_acc: 0.2570

In [21]:
dkt_model.fit([skill_response_array[:, 0:-1], masking_array[:, 1:]],
              response_array[:, 1:, np.newaxis],
              epochs=20, 
              batch_size=32, 
              shuffle=True,
              validation_split=0.2)

Train on 467 samples, validate on 117 samples
Epoch 1/20
467/467 [==============================] - 3s - loss: 0.6659 - acc: 0.8261 - val_loss: 0.6553 - val_acc: 0.8329

# Question 1
#### What were the 5 most common and 5 least common skills in this dataset? What percentage of responses are associated with the most common skill?

The 5 most common skills were: 87: Equation Solving Two or Fewer Steps, 30: Conversion of Fraction Decimals Percents, 71: Addition and Subtraction Fractions, 68: Addition and Subtraction Integers, 33: Ordering Fractions. 

The 5 least common skills were: 28: Reading a Ruler or Scale, 102: Recognize Quadratic Pattern, 98: Finding Slope from Ordered Pairs, 96: Finding Slope From Situation, 99: Distributive Property.

87 (Equation Solving Two or Fewer Steps) was the most common skill.

5.77% of the responses corresponded with this skill.

In [25]:
sorted_df = df.groupby(by=['skill_name']).count()

most = sorted_df.sort_values(by='order_id', ascending=[False]).index[0:5]
print("5 most common skills are:", [str(skill_dict[skill]) + ": " + skill for skill in most])

least = sorted_df.sort_values(by='order_id', ascending=[True]).index[0:5]
print("5 least common skills are:", [str(skill_dict[skill]) + ": " + skill for skill in least])

5 most common skills are: ['87: Equation Solving Two or Fewer Steps', '30: Conversion of Fraction Decimals Percents', '71: Addition and Subtraction Fractions', '68: Addition and Subtraction Integers', '33: Ordering Fractions']
5 least common skills are: ['28: Reading a Ruler or Scale', '102: Recognize Quadratic Pattern', '98: Finding Slope from Ordered Pairs', '96: Finding Slope From Situation', '99: Distributive Property']


In [5]:
most_df = sorted_df.sort_values(by='order_id', ascending=[False])
total = most_df.ix[:,0].sum()
most_common_skill = most_df.iloc[0,0]

most_common_skill / total

0.057708219855885375

# Question 2
#### Train the sequence prediction model using a randomly selected 70% (training set) of students' data and predict on the remaining 30% (test set). What was the overall accuracy of skill prediction in the test set? What were the top 5 hardest and easiest to predict skills? Describe the metric you chose to represent hard/easy prediction. 

In [24]:
from sklearn import model_selection
model_selection.train_test_split()